<a href="https://colab.research.google.com/github/OmarTarekAbdelWahab/Markov-Decision-Processes/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#**Assignment 3**
---
##**Markov Decision Processes**



*   Omar Tarek Abdelwahab       20010998
*   Abdelrahman Elsayed Mohamed 20010795
*   Marwan Mostafa Abd El-Kader 20011867
*   Ali Mones Abd El-Mohsen     20010946



In [ ]:
ROW_SIZE, COL_SIZE = 3, 3
GAMMA = .99

**MOVES**: array of possible moves at any cell

**directions**: used to print the policies in a prettier way

In [ ]:
MOVES = [(0,1),(0,-1),(1,0),(-1,0)]
directions = {
      (0,0):"Term",
      (0,1):"right",
      (0,-1):"left",
      (1,0):"down",
      (-1,0):"up"
}

Choose epsiolon value to be 1e-10
to end the iteration

In [ ]:
EPSILON = 0.0000000001

Get the next child states of the current state and current action.
return a dictionary with the state as key, and the probability of reaching that state as the value.


In [ ]:
def getNextVerMove(row, a_x):
  return min(max(row + a_x, 0), ROW_SIZE-1)
def getNextHozMove(col, a_y):
  return min(max(col + a_y, 0), COL_SIZE-1)
def addToDict(dic, v1, v2, val):
  if (v1, v2) not in dic:
    dic[(v1,v2)] = 0
  dic[(v1,v2)] += val
def getNextStates(row, col, action):
  a_x, a_y = action[0], action[1]
  nextStates = {}
  if a_y == 0:
    nextStates[(getNextVerMove(row, a_x), col)] = .8
    addToDict(nextStates, row,getNextHozMove(col, 1), .1)
    addToDict(nextStates, row,getNextHozMove(col, -1), .1)
  else:
    nextStates[(row, getNextHozMove(col, a_y))] = .8
    addToDict(nextStates, getNextVerMove(row,1), col, .1)
    addToDict(nextStates, getNextVerMove(row,-1), col, .1)
  return nextStates

function to calculate the relative error of the current value of V, copmuted by taking the max difference of the values in the current V array and the previous one.


In [ ]:
def calculateRelativeError(prev, next):
  diff = 0
  for i in range(ROW_SIZE):
    for j in range(COL_SIZE):
      if i==0 and (j==0 or j == 2):
        continue
      diff = max(diff,abs(prev[i][j] - next[i][j]))
  return diff

In [ ]:
def get_copy(prev, next):
  for i in range(ROW_SIZE):
    for j in range(COL_SIZE):
      prev[i][j] = next[i][j]

function to implement the value iteration according to the Bellman equation:
> **V(s)=max a: ∑T(s,a,s′)​(R(s,a,s′)+γV(s′))**





In [ ]:
def value_iteration(reward):
  V_0 = [[0,0,0],[0,0,0],[0,0,0]]
  V_1 = [[0,float('-inf'),0],
        [float('-inf'),float('-inf'),float('-inf')],
        [float('-inf'),float('-inf'),float('-inf')]
          ]
  curr_error = 1
  cnt = 0
  while curr_error > EPSILON:
    cnt += 1
    for i in range(ROW_SIZE):
      for j in range(COL_SIZE):
        if i==0 and (j==0 or j == 2):
          continue
        for move in MOVES:
          sum = 0
          nextStates = getNextStates(i,j,move)
          for nextState in nextStates:
            sum += nextStates[nextState]*(reward[nextState[0]][nextState[1]]+GAMMA*V_0[nextState[0]][nextState[1]])
          V_1[i][j] = max(V_1[i][j], sum)
    curr_error = calculateRelativeError(V_0, V_1)
    get_copy(V_0, V_1)
  return cnt, V_0

function to extract the policy, done once after the value iteration.


In [ ]:
def policy_extraction(V, reward):
  policy = [[(0,0),(0,0),(0,0)],[(0,0),(0,0),(0,0)],[(0,0),(0,0),(0,0)]]
  for i in range(ROW_SIZE):
    for j in range(COL_SIZE):
      if i==0 and (j==0 or j == 2):
        continue
      mx = float('-inf')
      for move in MOVES:
        sum = 0
        nextStates = getNextStates(i,j,move)
        for nextState in nextStates:
          sum += nextStates[nextState]*(reward[nextState[0]][nextState[1]]+GAMMA*V[nextState[0]][nextState[1]])
        if sum > mx:
          mx = sum
          policy[i][j] = move
  return policy

Reset the variables.

In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


print the states with the policies.

In [ ]:
def print_policy(policy):
  for i in range(ROW_SIZE):
    for j in range(COL_SIZE):
      print(directions[policy[i][j]], end = " ")
    print("")

Implement the policy iteration, according to the function:
> **VΠ(s)=max a: ∑T(s,Π(s),s′)​(R(s,Π(s),s′)+γVΠ(s′))**

In [ ]:
def policy_iteration(reward, policy):
  V_0 = [[0,0,0],[0,0,0],[0,0,0]]
  V_1 = [[0, float('-inf'), 0],
        [float('-inf'), float('-inf'), float('-inf')],
        [float('-inf'), float('-inf'), float('-inf')]]
  curr_error = 1
  cnt = 0
  while curr_error > EPSILON:
    cnt += 1
    for i in range(ROW_SIZE):
      for j in range(COL_SIZE):
        if i==0 and (j==0 or j == 2):
          continue
        move = policy[i][j]
        sum = 0
        nextStates = getNextStates(i,j,move)
        for nextState in nextStates:
          sum += nextStates[nextState]*(reward[nextState[0]][nextState[1]]+GAMMA*V_0[nextState[0]][nextState[1]])
        V_1[i][j] = sum
    curr_error = calculateRelativeError(V_0, V_1)
    get_copy(V_0, V_1)
  return cnt, V_0

call the policy iteration then the extraction, break when the policy isn't changing anymore.


In [ ]:
def policy_evaluation(reward, policy):
  cnt = 0
  V = []
  while True:
    cnt+=1
    iterations, V = policy_iteration(reward, policy)
    new_policy = policy_extraction(V, reward)
    if policy == new_policy:
      break
    policy = new_policy
  return cnt, policy, V


Call both approaches with


```
r = [-3, 0, 3, 100]
```

In [31]:
# value iteration:
for r in [100, 3, 0, -3]:
  temp_policy = [[(0,1),(0,1),(0,1)],[(0,1),(0,1),(0,1)],[(0,1),(0,1),(0,1)]]
  print("r =",r)
  reward = [[r,-1,10],[-1,-1,-1],[-1,-1,-1]]
  iterations, V = value_iteration(reward)
  print("Finished value iterations in ",iterations, "iterations")
  print("V =",V)
  print("policy after the policy extraction:")
  print_policy(policy_extraction(V, reward))
  print()
  iterations, policy, V = policy_evaluation(reward, temp_policy)
  print("Finished Policy evaluation in ", iterations, "iterations")
  print("V =",V)
  print("Policy:")
  print_policy(policy)
  print(end = "\n\n\n")

r = 100
Finished value iterations in  44 iterations
V = [[0, 99.19548076040549, 0], [99.19548076040549, 96.71846631440145, 90.10531413789691], [96.44629713553942, 94.2958884533505, 91.67898973883231]]
policy after the policy extraction:
Term left Term 
up left down 
up left left 

Finished Policy evaluation in  4 iterations
V = [[0, 99.19548076040545, 0], [99.19548076040545, 96.71846631440124, 90.10531413789214], [96.44629713553917, 94.29588845334996, 91.67898973883001]]
Policy:
Term left Term 
up left down 
up left left 



r = 3
Finished value iterations in  39 iterations
V = [[0, 9.557514418221283, 0], [6.448045509276803, 8.195156472913931, 9.557514418221283], [5.631127098398129, 6.862738649324498, 8.045463424538513]]
policy after the policy extraction:
Term right Term 
right right up 
right right up 

Finished Policy evaluation in  3 iterations
V = [[0, 9.557514418220777, 0], [6.4480455092711075, 8.195156472911938, 9.557514418220777], [5.63112709838591, 6.8627386493191835, 8.045463

With r = 100 we got the policy:

```
Term left Term
up left down
up left left
```
***where Term means a terminal state.***

All cells are following the intuitive path to the cell with the highest value ( r ), except for the last cell in the second row with the ( **down** ) move, the reason fot this move is that it wants to avoid the 10% chance to go up to the other terminal state with the value 10, where the game would also end.

---

With r = 3 we got the policy:
```
Term right Term
right right up
right right up
```

All the cells are trying to reach the topright cell with the highest reward of 10, notice that the first cell in the second row did't move down like the previous example, meaning that the 10% chance to get a reward of 3 is not as bad, and going around the whole grid to reach the 10 goal might not be worth it.

---

With r = 0 er got the policy:
```
Term right Term
right right up
right right up
```

Which is the same as r = 3.

---
With r = -3 we got the policy:

```
Term right Term
right right up
right right up
```

Still -3 is not enough to get the policy to avoid it.
Using binary search, we found out that -16 is the max value at which it considers to go down and around.
```
r = -16
Term right Term
down right up
right right up
```